In [1]:
from SBM_SDE_tensor import *
from obs_and_flow import *
from mean_field import *
from training import *
import torch
#import seaborn as sns

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .1 #SDE discretization timestep.
t = 200 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
pretrain_lr = 1e-3
elbo_lr = 1e-3
niter = 250
piter = 50
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.
prior_scale_factor = 1 / 3 #Prior standard deviations set at 0.25 of prior means.
obs_error_scale_factor = 1 / 3

In [3]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [4]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [5]:
net1, q_theta1, ELBO_losses1, norm_losses1 = train(devi, pretrain_lr, elbo_lr, niter, piter, batch_size, prior_scale_factor, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_prior_means, temp_gen, temp_ref, analytical_steady_state_init_CON)



Train Diffusion:   0%|          | 0/250 [00:00<?, ?it/s]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.9150,  1.3324,  0.5043],
         [ 2.2110,  0.9742,  0.5536],
         ...,
         [ 0.8584,  0.9741,  0.4525],
         [ 0.6547,  0.7438,  0.7634],
         [ 0.6937,  0.4397,  0.7510]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.9150,  0.5253,  1.0934],
         [ 1.0510,  1.4237,  1.0289],
         ...,
         [ 0.7520,  0.3948,  1.0148],
         [ 1.0334,  1.5046,  0.6201],
         [ 0.9632,  1.0478,  0.5687]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[1.1851, 0.8622, 0.7119],
        [1.1968, 0.8584, 0.7057]], grad_fn=<MeanBackward1>)

Moving average norm loss at <built-in function iter> iterations is: 2811.585693359375. Best norm loss value is: 2811.585693359375.

C_PATH mean = tensor([[1.1851, 0.8622, 0.7119],
        [1.1968, 0.8584, 0.7057]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],



Train Diffusion:   0%|          | 1/250 [00:01<07:56,  1.92s/it]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 2.0177,  0.5623,  0.6445],
         [ 3.3691,  0.8018,  0.7963],
         ...,
         [ 2.4934,  0.6269,  0.5420],
         [ 1.2382,  0.7503,  1.4173],
         [ 1.7918,  2.1428,  0.7742]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.8263,  0.9327,  0.5807],
         [ 0.6409,  0.9184,  0.8997],
         ...,
         [ 0.8075,  0.7010,  0.4244],
         [ 0.6313,  1.3949,  0.7361],
         [ 0.8408,  0.5589,  0.6331]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[1.8428, 0.8658, 0.6135],
        [1.8068, 0.8565, 0.5937]], grad_fn=<MeanBackward1>)




Train Diffusion:   1%|          | 2/250 [00:03<07:48,  1.89s/it]

...



Train Diffusion:  20%|██        | 50/250 [01:36<05:52,  1.76s/it]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [2.2134e+00, 1.1859e+01, 3.2666e-02],
         [3.1392e-02, 1.2885e+01, 3.0226e-02],
         ...,
         [1.8862e+01, 4.1321e-02, 3.1387e-02],
         [1.8282e+01, 4.5417e-02, 8.9819e-02],
         [1.4216e+01, 4.1771e-02, 6.5507e-01]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [1.0659e+01, 9.0339e-01, 4.6943e-02],
         [2.3258e+01, 3.1090e+00, 5.5762e-02],
         ...,
         [5.5823e-02, 4.6915e-02, 3.0313e-02],
         [4.8365e-01, 4.0330e-02, 6.7552e-02],
         [8.3043e-01, 4.0871e-02, 6.2903e-01]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[20.4846,  0.1457,  0.0415],
        [20.3812,  0.2128,  0.0414]], grad_fn=<MeanBackward1>)

Moving average norm loss at <built-in function iter> iterations is: 2045.3388793945312. Best norm loss value is: 2013.93701171875.

C_PATH mean = tensor([[20.4846,  0.1457, 



Train Diffusion:  20%|██        | 51/250 [01:38<06:25,  1.94s/it]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [2.2176e+00, 5.5059e-01, 3.9024e-02],
         [3.4320e-02, 4.9311e+00, 4.4950e-02],
         ...,
         [3.1583e+01, 4.7080e-02, 2.6088e-02],
         [4.4322e-01, 3.8445e-02, 6.2218e-02],
         [4.7259e-01, 4.1555e-02, 4.7806e-01]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [1.0534e+01, 1.7618e+01, 3.9792e-02],
         [2.3401e+01, 1.9724e+01, 3.5346e-02],
         ...,
         [3.1405e-02, 3.2162e-02, 2.2662e-02],
         [2.6122e+01, 4.1982e-02, 5.3593e-02],
         [1.9560e+01, 3.9642e-02, 5.2552e-01]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[19.8030,  0.3088,  0.0367],
        [21.5276,  0.3125,  0.0364]], grad_fn=<MeanBackward1>)


NameError: name 'dt' is not defined

In [ ]:
net2, q_theta2, ELBO_losses2, norm_losses2 = train(devi, pretrain_lr, elbo_lr, niter, piter, batch_size, prior_scale_factor, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_prior_means, temp_gen, temp_ref, analytical_steady_state_init_CON, LEARN_THETA = True)